In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import pytorch_lightning as pl
import glob

## First, we slice the train images into 31 x 31 pixels with the ground truth in the middle

In [16]:
def ndigit(n, x):
    x = str(x)
    while(len(x) < n):
        x = "0" + x
    return x

In [17]:
def load_data(res, files = 20):
    j = 0
    path = ["02", "train"]
    res = int((res-1)/2)
    nan_values = 0
    for p in path:
        for f in range(files):
            image = np.load(f"images_{p}/images/image_{ndigit(3, f)}.npy")
            mask = np.load(f"masks_{p}/masks/mask_{ndigit(3, f)}.npy")
            image = np.reshape(image, (1024,1024,10))
            mask = np.reshape(mask, (1024,1024,1))
            
            nan_values_before = (np.count_nonzero(np.isnan(image)))
            
            channel8 = image[:, :, 7]
            channel4 = image[:, :, 3]
            channels = image.shape
            width = image[0].shape[0]
            height = image[0].shape[1]

            # add the vegetation array 
            vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(np.add(channel8, channel4), .0001))
            vegetation_array = np.nan_to_num(vegetation_array, nan=0.0)
            image_transformed = np.concatenate((image, vegetation_array[:, :, np.newaxis]), axis=2)

        
            image = image_transformed
            nan_values_before = 0
            # add moisture index
            channel8a = image[:, :, 7]
            channel11 = image[:, :, 8]
            moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(np.add(channel8a, channel11), .0001))
            
            nan_values_moisture = (np.count_nonzero(np.isnan(moisture_array)))
            
            if(nan_values_moisture > 0): 
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy had", nan_values_before, "before moisture index")
                print("picture",f"images_{p}/images/image_{ndigit(3, f)}.npy has", nan_values_moisture, "nan_values after adding moisture")
            
            # fill nan values with 0
            moisture_array = np.nan_to_num(moisture_array, nan=0.0)
            
            image_transformed = np.concatenate((image,moisture_array[:,:, np.newaxis]), axis = 2)
            image = image_transformed

            nan_values_pic = np.count_nonzero(np.isnan(image))
            nan_values += nan_values_pic
            # Add padding to every image (and mask) edge in case there are ground truths which are too close to an edge
            padded_image = np.pad(image, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')
            padded_mask = np.pad(mask, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')

            # Extract ground truths
            ground_truths_pos = np.array(np.where(padded_mask != 0)).T
            
            # Slice and save patches around each ground truth
            for i in ground_truths_pos: 
                patch = (padded_image[i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1, :], padded_mask[i[0], i[1], 0])
                np.save(f"patches/train/patch_{p}_{ndigit(3, f)}_{ndigit(5, j)}.npy", np.array(patch, dtype="object"))                                 
                j += 1
    print("Added Vegetation (B8-B4)/(B8+B4)")
    print("Added Moisture (B8A-B11)/(B8A+B11)")
    print("Patched the pictures")
    print("NaN values:", nan_values)


In [18]:
# Check number of ground truths
# pos = 0
# for i in range(20):
#     mask = np.load(f"masks_02/masks/mask_{ndigit(3, i)}.npy")
#     ground_truths_pos = np.array(np.where(mask != 0)).T
#     pos = pos + len(ground_truths_pos)
# print(pos)

In [19]:
res = 15
load_data(res)

Added Vegetation (B8-B4)/(B8+B4)
Added Moisture (B8A-B11)/(B8A+B11)
Patched the pictures
NaN values: 0


## Then, we load the data and have a look

In [20]:
batch_size = 3

In [21]:
# Load patches
directory = 'patches/train'
file_paths = glob.glob(directory + '/*.npy')
trainset0 = [np.load(file_path, allow_pickle=True) for file_path in file_paths]
trainset = []
for pic in trainset0:
    trainset.append(pic)
len(trainset)

KeyboardInterrupt: 

In [ ]:
trainset[0]

In [ ]:
trainset_enriched = trainset
print(trainset_enriched[0])
#len(trainset_enriched)
#trainset[pic_no][0][h][w][channel] -> pixel value
#trainset[pic_no][1] -> Ground truth 

In [ ]:
X,y = trainset_enriched[0]
X.shape
trainset_enriched

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, trainset, transform):
        self.trainset = trainset
        self.transform = transform
       #self.augmentations = augmentations

    def __len__(self):
        return len(self.trainset)

    def __getitem__(self, index):
        data, target = self.trainset[index]

        # apply each transformation jointly to each input
        if self.transform:
            data = self.transform(data)

        # apply each augmentation separately to each input
        #if self.augmentations:
        #    for augmentation in self.augmentations:
        #        data = augmentation(data)


        return data, target

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.ConvertImageDtype(torch.float64),
     transforms.Lambda(lambda x : x / 3000),
     transforms.Lambda(lambda x : torch.where(x > 1, 1, x)), # clip images between 0 and 1
     transforms.Normalize(mean=(0.5,)*12,
                          std=(0.5,)*12)
     ])

# augmentations = [
#      transforms.RandomRotation(360),
#      transforms.RandomAffine(degrees=0, translate=(0.5,0.5)), # shift in both directions along 0.5 * height on y-axis and 0.5 * width on x-axis
#      transforms.RandomAffine(0, scale=(10,45)), # scale in range 10 <= scale <= 45
#      transforms.RandomAffine(0, shear=[10,30,10,30]), # shear on x- and y-axis between (10,30) 
#      transforms.ElasticTransform(alpha=50.0, sigma=3.0) # displaces pixels
# ]

In [ ]:
# Create the custom dataset
trainset_transformed = CustomDataset(trainset_enriched, transform=transform)
trainset_transformed[0][0]
#on the fly augmentation during training, hence no additional pictures in trainset 
#len(trainset_transformed)

In [13]:
#trainset_transformed[0]

In [14]:
# Calculate the sizes of the training set and validation set
train_size = int(0.8 * len(trainset_transformed))
val_size = len(trainset_transformed) - train_size

# Split trainset into trainset and valset
trainset_load, valset_load = random_split(trainset_transformed, [train_size, val_size])
print(len(trainset_load), len(valset_load))

# Create data loaders for the training set and validation set
trainloader = DataLoader(trainset_load, batch_size=batch_size, shuffle=True)
validloader = DataLoader(valset_load, batch_size=batch_size, shuffle=False)


31090 7773


In [15]:
limit = 0
for batch in trainloader:

    while(limit <1):
        print(batch)
    #    print(x.shape)
    #    print(x.size)
    #    print(y.shape)
    #    print(y)
        limit += 1

[tensor([[[[-0.2820, -0.2540, -0.0807,  ...,  0.3527,  0.3093, -0.0673],
          [-0.2800, -0.3913, -0.4380,  ...,  0.2993,  0.3813,  0.4567],
          [-0.2527, -0.4520, -0.3493,  ...,  0.3893,  0.4527,  0.4207],
          ...,
          [-0.3187, -0.2667, -0.2860,  ...,  0.4493,  0.7293,  0.4833],
          [-0.4253, -0.4093, -0.3647,  ...,  0.5773,  0.7640,  0.5827],
          [-0.2133, -0.3687, -0.2640,  ...,  0.3087,  0.8207,  0.4447]],

         [[-0.2820, -0.2540, -0.0807,  ...,  0.3493,  0.3147,  0.1487],
          [-0.2720, -0.4233, -0.4173,  ...,  0.2993,  0.3813,  0.4567],
          [-0.2527, -0.4520, -0.3493,  ...,  0.3893,  0.4093,  0.4333],
          ...,
          [-0.3187, -0.2667, -0.2860,  ...,  0.4493,  0.7293,  0.4833],
          [-0.4093, -0.3833, -0.3893,  ...,  0.5267,  0.6480,  0.6233],
          [-0.3553, -0.3620, -0.1593,  ...,  0.3087,  0.8207,  0.4447]],

         [[-0.2460, -0.3233, -0.1553,  ...,  0.3493,  0.3147,  0.1487],
          [-0.2720, -0.4233, 

## Next, we define the model and train it

In [ ]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

    def __init__(self, *layers, classes=None):
        super().__init__()

        self.lr = 0.01  # Assign the learning rate here
        self.classes = classes

        self.layers = nn.Sequential(*layers)  # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)

    def predict(self, X):
        with torch.no_grad():
            y_hat = self(X).argmax(1)
        if self.classes is not None:
            y_hat = [self.classes[i] for i in y_hat]
        return y_hat

    def training_step(self, batch, batch_idx, log_prefix='train'):
        X, y = batch
        print(X[0].shape, "X")
        print(y[0].shape, "y")
        y_hat = self(X)
        if(y_hat == None):
            return 0
        loss_fn = nn.MSELoss()
        loss = loss_fn(y_hat, y)
        #print(y_hat, "yhat")
        print(y_hat) #[[3,1 5]] 7]] 5]]
        #print(y_hat.size)
        #print(y_hat.shape)
        self.log(f"{log_prefix}_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            return self.training_step(batch, batch_idx, log_prefix='valid')

    def configure_optimizers(self):
        # Adam with Weight Decay
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

        # Simplest scheduler is ReduceLROnPlateau. This scheduler reduces the learning rate by 0.1
        # if the val_loss has not decreased within the last 10 epochs.
        scheduler = {
            # REQUIRED: The scheduler instance
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "epoch",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
            # Metric to to monitor for schedulers like `ReduceLROnPlateau`
            "monitor": "valid_loss",
            # If set to `True`, will enforce that the value specified 'monitor'
            # is available when the scheduler is updated, thus stopping
            # training if not found. If set to `False`, it will only produce a warning
            "strict": True,
            # If using the `LearningRateMonitor` callback to monitor the
            # learning rate progress, this keyword can be used to specify
            # a custom logged name
            "name": None,
        }
        return {"optimizer": optimizer, 'lr_scheduler': scheduler}

## Implement model

In [ ]:
# Implements entry to SepConv2d, see Lang et al. (2019), p. 6
class MyEntryLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.out_channels = out_channels

        self.proj_out = nn.Conv2d(in_channels, out_channels[len(out_channels)-1], (1,1))

        self.entry_blocks = nn.ModuleList()
        for i in range(len(out_channels)):
            self.entry_blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels[i], (1, 1)),
                nn.BatchNorm2d(out_channels[i]),
                nn.ReLU(inplace = True)
            ))
            in_channels = out_channels[i]  # Update in_channels for next iteration

    def forward(self, x):
        x_entry = x
        for i in range(len(self.out_channels)):
            x_entry = self.entry_blocks[i](x_entry)
        x = self.proj_out(x)
        return (x + x_entry)

In [ ]:
# Implements SepConv2D
class MySepConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)

        self.sep_conv_block = nn.Sequential(
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels, in_channels, kernel, groups=in_channels, **kwargs), # depthwise SepConv
            nn.Conv2d(in_channels, out_channels, (1,1), **kwargs), # pointwise SepConv
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        x_sep_conv = self.sep_conv_block(x)
        x_sep_conv_2 = self.sep_conv_block(x_sep_conv) # performs second SepConv, see Lang et al. (2019), p. 6
        x = self.proj_out(x)
        return (x + x_sep_conv_2) # adds original input and sep_conv_2 output

In [ ]:
tree_model = MyCNNModel(
    #MyEntryLayer(12, [128, 256]), # increase number of channels to 512
    nn.Conv2d(12, 256, (3,3), padding='same'),
    #MySepConvLayer(256, 256, (3,3), padding='same'),
    #MySepConvLayer(256, 256, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(256, 1)
)

In [ ]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='64', max_epochs=1,
                      callbacks=[RichProgressBar(refresh_rate=50),
                                 RichModelSummary(3),
                                ])

In [ ]:
trainer1.fit(tree_model, trainloader, validloader)

In [ ]:
tree_model.eval()
tree_model = tree_model.float()
batch = next(iter(trainloader))
inputs = batch[0]
inputs = inputs.float()

print(inputs.shape)


with torch.no_grad():
    predictions = tree_model(inputs)


print("Predictions:", predictions)
# expected 128 pictures, each with 15 * 15 predictions for height ! 